In [17]:
%load_ext bigdata
%hive_start
%timeout 300

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [18]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 14.653 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.112 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.799 seconds
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.25 seconds
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.098 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 1.107 seconds


In [73]:
%%hive
SELECT t.* 
FROM (SELECT YEAR(c4),explode(c5) FROM tbl0) t;

SELECT t.* 
FROM (SELECT YEAR(c4),explode(c5) FROM tbl0) t;
FAILED: SemanticException [Error 10081]: UDTF's are not supported outside the SELECT clause, nor nested in expressions


In [58]:
%%hive
SHOW TABLES;

SHOW TABLES;
OK
tbl0
tbl1
Time taken: 0.065 seconds, Fetched: 2 row(s)


In [66]:
%%hive
SELECT DISTINCT t.* 
FROM (SELECT c4,explode(c5) FROM tbl0) t;

SELECT DISTINCT t.* 
FROM (SELECT c4,explode(c5) FROM tbl0) t;
FAILED: SemanticException [Error 10081]: UDTF's are not supported outside the SELECT clause, nor nested in expressions


In [76]:
%%hive
SHOW TABLES;

SHOW TABLES;
OK
let_by_year
tbl0
tbl1
Time taken: 0.061 seconds, Fetched: 3 row(s)


In [81]:
%%hive
SELECT explode(c5) from tbl0;

SELECT c4,explode(c5) from tbl0;
FAILED: SemanticException [Error 10081]: UDTF's are not supported outside the SELECT clause, nor nested in expressions


In [3]:
%%hive
SELECT t.c2, concat_ws(':', collect_set(CAST(t.c1 AS VARCHAR(10))))
FROM (SELECT c2, c1
FROM tbl0) t
GROUP BY t.c2;

SELECT t.c2, concat_ws(':', collect_set(CAST(t.c1 AS VARCHAR(10))))
FROM (SELECT c2, c1
FROM tbl0) t
GROUP BY t.c2;
Query ID = root_20200306134755_f370ea54-bdce-41f7-b7d0-0657f9e2408c
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1583494662535_0019, Tracking URL = http://a72bcf2055fd:8088/proxy/application_1583494662535_0019/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1583494662535_0019
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-03-06 13:48:10,590 Stage-1 map = 0%,  reduce = 0%
2020-03-06 13:48:17,751 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 4.63 sec
2020-0

In [22]:
%hive_quit

In [21]:
%%hive
--
-- Escriba una consulta que retorne la columna `tbl0.c1` y el valor 
-- correspondiente de la columna `tbl1.c4` para la columna `tbl0.c2`.
--
-- Escriba el resultado a la carpeta `output` de directorio de trabajo.
--
SELECT t0.c1, t0.c2, t1.v
FROM
(SELECT c1,c2 FROM tbl0) t0 LEFT JOIN
(SELECT c1, k,v FROM tbl1 LATERAL VIEW
EXPLODE(c4) tbl1 as k,v) t1 ON (t0.c1 = t1.c1 AND t0.c2 = t1.k);

--
-- Escriba una consulta que retorne la columna `tbl0.c1` y el valor 
-- correspondiente de la columna `tbl1.c4` para la columna `tbl0.c2`.
--
-- Escriba el resultado a la carpeta `output` de directorio de trabajo.
--
SELECT t0.c1, t0.c2, t1.v
FROM
(SELECT c1,c2 FROM tbl0) t0 LEFT JOIN
(SELECT c1, k,v FROM tbl1 LATERAL VIEW
EXPLODE(c4) tbl1 as k,v) t1 ON (t0.c1 = t1.c1 AND t0.c2 = t1.k);
Query ID = root_20200306144715_dd979292-8318-4bec-ad9b-4a62f42ec3f7
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2020-03-06 14:47:28	Starting to launch local t